# Running inference at scale and in realtime


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/computer-vision/deeplearning-cv-pcb-flow-5.png?raw=true" width="700px" style="float: right"/>

We have now deployed our model to our Registry. The registry provides governance and ACL, simplifying and accelerating all downstream pipeline developments.

Other teams don't have to worry about the model itself, they can focus on Ops task & model serving, while Data Scientists can release new models when then feel ready. 

Models are typically used in 2 ways:

- At scale, a cluster (in batch or streaming, including within Delta Live Tables)
- For realtime, low-latencies use-cases, served behind a REST API.

Databricks provides and simplify both options.

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1549883858499596&notebook=%2F03-running-cv-inferences&demo_name=computer-vision-pcb&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fcomputer-vision-pcb%2F03-running-cv-inferences&version=1">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-computer-vision-pcb-shawnzou2020` from the dropdown menu ([open cluster configuration](https://dbc-abdbb8e0-f50f.cloud.databricks.com/#setting/clusters/0410-040209-qt57lxc5/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('computer-vision-pcb')` or re-install the demo: `dbdemos.install('computer-vision-pcb')`*

## Batch & Streaming Model scoring

Let's start with Batch / Streaming inference. We'll use spark distributed capabilities to run our inferences at scale.

To do so, we need to load our model from the MLFlow registry, and build a Pandas UDF to distribute the inference on multiple instances (typically on GPUs).

The first step consist on installing the model dependencies to make sure we're loading the model using the same librairies versions.

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository
import os
requirements_path = ModelsArtifactRepository("models:/dbdemos_pcb_classification/Production").download_artifacts(artifact_path="requirements.txt") # download model requirement from remote registry

if not os.path.exists(requirements_path):
  dbutils.fs.put("file:" + requirements_path, "", True)

In [0]:
%pip install -r $requirements_path

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 42.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ./_resources/00-init $reset_all_data=false $db=dbdemos $catalog=manufacturing_pcb

USE CATALOG `manufacturing_pcb`
using cloud_storage_path /Users/quentin.ambard@databricks.com/demos/manufacturing_pcb_dbdemos
using catalog.database `manufacturing_pcb`.`dbdemos`


data already existing. Run with reset_all_data=true to force a data cleanup for your local demo.


In [0]:
import torch
#Make sure to legerage the GPU when available
model_uri = "models:/dbdemos_pcb_classification/Production"
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
pipeline = mlflow.transformers.load_model(model_uri, device=device.index)

2024/03/04 22:09:15 INFO mlflow.transformers: 'models:/dbdemos_pcb_classification/Production' resolved as 'dbfs://databricks/databricks/mlflow-registry/329ad5292e104cf187af92453f5bfa46/models/model'
/databricks/python/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


### Running our model inference locally (non distributed)

Let's first run prediction locally on a standard pandas dataframe:

In [0]:
import io
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image

#Call the pipeline and returns the main class with its probability
def predict_byte_series(content_as_byte_series, pipeline):
  #Transform as a list of PIL Images for our huggingface pipeline:
  image_list = content_as_byte_series.apply(lambda b: Image.open(io.BytesIO(b))).to_list()
  #the pipeline returns the probability for all the class
  predictions = pipeline.predict(image_list)
  #Filter & returns only the class with the highest score [{'score': 0.999038815498352, 'label': 'normal'}, ...]
  return pd.DataFrame([max(r, key=lambda x: x['score']) for r in predictions])  


df = spark.read.table("training_dataset_augmented").limit(50)
#Switch our model in inference mode
pipeline.model.eval()
with torch.set_grad_enabled(False):
  predictions = predict_byte_series(df.limit(10).toPandas()['content'], pipeline)
display(predictions)

score,label
0.9999707937240601,normal
0.9998716115951538,normal
0.9999814033508301,normal
0.9999306201934814,normal
0.999972939491272,normal
0.9995645880699158,damaged
0.9999706745147705,normal
0.9999879598617554,normal
0.9979193806648254,normal
0.9999420642852783,normal


### Distribute the inference with Spark and a Pandas UDF (batch/streaming inference)
 
Let's parallelize our inferences by wrapping the function using a pandas UDF:

In [0]:
import numpy as np
import torch
from typing import Iterator

#Only batch the inferences in our udf by 1000 as images can take some memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1000)
@pandas_udf("struct<score: float, label: string>")
def detect_damaged_pcb(images_iter: Iterator[pd.Series]) -> Iterator[pd.DataFrame]:
  #device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
  #pipeline = mlflow.transformers.load_model(model_uri, device=device.index) #Load pipeline outside for now to avoid error (ES-750064)
  #Switch pipeline to eval mode
  pipeline.model.eval()
  with torch.set_grad_enabled(False):
    for images in images_iter:
      yield predict_byte_series(images, pipeline)

In [0]:
display(df.select('filename', 'content').withColumn("prediction", detect_damaged_pcb("content")))

filename,content,prediction
0930.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9999708, normal)"
0568.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9998716, normal)"
0551.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9999814, normal)"
0261.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9999306, normal)"
0127.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.99997294, normal)"
044.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9995646, damaged)"
0898.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9999707, normal)"
0160.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.99998796, normal)"
0677.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.9979194, normal)"
0234.JPG,"List(/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjI= (truncated), ...)","List(0.99994206, normal)"


## Realtime inferences with REST API

Many use-case requires real-time capabilities. Think about realtime analysis in our PCB manufacturing system. A picture is taken and we need to instantly check for potential defects. 

To be able to do that, we'll need to serve our inference behind a REST API. The system can then send our images, and the endpoint answer with the prediction.

To do that, we'll have to encode our image byte as base64 and send them over the REST API.

This implies that our model should decode the base64 back as PIL image. To make this easy, we can create a custom model wrapper having the transformers pipeline and a simple method transforming the base64 before calling our pipeline.

This is done as usual extending the `mlflow.pyfunc.PythonModel` class:

In [0]:
from io import BytesIO
import base64

# Model wrapper
class RealtimeCVModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, pipeline):
        self.pipeline = pipeline
        # instantiate model in evaluation mode
        self.pipeline.model.eval()

    #images will contain a series of images encoded in b64
    def predict(self, context, images):
        with torch.set_grad_enabled(False):
          #Convert the base64 to PIL images
          images = images['data'].apply(lambda b: Image.open(BytesIO(base64.b64decode(b)))).to_list()
          predictions = self.pipeline(images)
          return pd.DataFrame([max(r, key=lambda x: x['score']) for r in predictions])
        
#Build our final hugging face pipeline. Load it as CPU as our endpoint will be cpu for now
pipeline_cpu = mlflow.transformers.load_model(model_uri, return_type="pipeline", device=torch.device("cpu").index)
rt_model = RealtimeCVModelWrapper(pipeline_cpu)

def to_base64(b):
  return base64.b64encode(b).decode("ascii")

#Let's try locally before deploying our endpoint to make sure it works as expected:
pdf = df.toPandas()

#Transform our input as a pandas dataframe containing base64 as this is what our serverless model endpoint will receive.
df_input = pd.DataFrame(pdf["content"].apply(to_base64).to_list(), columns=["data"])
predictions = rt_model.predict(None, df_input)
display(predictions)

2024/03/04 22:09:39 INFO mlflow.transformers: 'models:/dbdemos_pcb_classification/Production' resolved as 'dbfs://databricks/databricks/mlflow-registry/329ad5292e104cf187af92453f5bfa46/models/model'
/databricks/python/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


score,label
0.9999707937240601,normal
0.9998716115951538,normal
0.9999814033508301,normal
0.9999306201934814,normal
0.999972939491272,normal
0.9995645880699158,damaged
0.9999706745147705,normal
0.9999879598617554,normal
0.9979193806648254,normal
0.9999420642852783,normal


Now that our wrapper is ready, let's deploy it as a new model in the registry.

If realtime serving is your main usage, you would typically do that in the training step while registring your model. For this demo, we'll make 2 separate models: 1 for batch and 1 with the base64 wrapper for realtime inferences.

In [0]:
from mlflow.models.signature import infer_signature
init_experiment_for_batch("computer-vision-dl", "pcb")

with mlflow.start_run(run_name="hugging_face_rt") as run:
  signature = infer_signature(df_input, predictions)
  #log the model to MLFlow
  reqs = mlflow.pyfunc.log_model("model", python_model = rt_model, pip_requirements=requirements_path, input_example=df_input, signature = signature)
  mlflow.set_tag("dbdemos", "pcb_classification")
  mlflow.set_tag("rt", "true")

#Save the model in the registry & move it to Production
model_registered = mlflow.register_model("runs:/"+run.info.run_id+"/model", "dbdemos_pcb_classification")

Using common experiment under /Shared/dbdemos/experiments/computer-vision-dl/pcb


/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'dbdemos_pcb_classification' already exists. Creating a new version of this model...
2024/03/04 22:10:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: dbdemos_pcb_classification, version 25
Created version '25' of model 'dbdemos_pcb_classification'.


### Deploying a Model Endpoint Serving

Our new model wrapper is available in the registry. We can deploy this new version as a model endpoint to start out realtime model serving.


In [0]:
#Simple wrapper on top of the REST API. See the _resource/00-init companion notebook for more details or the Databricks endpoint API documentation
serving_client = EndpointApiClient()

# Start the endpoint using the REST API (you can do it using the UI directly)
# The first run might take some time as it's building the image. Further deployment are very fast even when scaled to zero.
serving_client.create_endpoint_if_not_exists("dbdemos_pcb_classification_endpoint", model_name="dbdemos_pcb_classification", model_version = model_registered.version, workload_size="Small", scale_to_zero_enabled=True, wait_start = True)

Current model is version 23. Updating to 25...
{"name":"dbdemos_pcb_classification_endpoint","creator":"quentin.ambard@databricks.com","creation_timestamp":1687197241000,"last_updated_timestamp":1709590223000,"state":{"ready":"READY","config_update":"IN_PROGRESS"},"config":{"served_models":[{"name":"dbdemos_pcb_classification-23","model_name":"dbdemos_pcb_classification","model_version":"23","workload_size":"Small","scale_to_zero_enabled":true,"workload_type":"CPU","state":{"deployment":"DEPLOYMENT_READY","deployment_state_message":""},"creator":"quentin.ambard@databricks.com","creation_timestamp":1709582809000}],"served_entities":[{"name":"dbdemos_pcb_classification-23","entity_name":"dbdemos_pcb_classification","entity_version":"23","workload_size":"Small","scale_to_zero_enabled":true,"workload_type":"CPU","state":{"deployment":"DEPLOYMENT_READY","deployment_state_message":""},"creator":"quentin.ambard@databricks.com","creation_timestamp":1709582809000}],"traffic_config":{"routes":[{

### Our endpoint is ready

You can access and configure your endpoint using the [Model Serving UI](/#mlflow/endpoints/dbdemos_pcb_classification_endpoint) or the API. The Model Endpoint is serverless, stopping and starting almost instantly. In our case, we chose to scale it down to zero when not used (ideal for test/dev environements). 


*Note that Databricks Model Serving lets you host multiple model versions, simplifying A/B testing and new model deployment.*

In [0]:
import timeit

endpoint_url = f"{serving_client.base_url}/realtime-inference/dbdemos_pcb_classification_endpoint/invocations"
print(f"Sending requests to {endpoint_url}")
for i in range(3):
    rest_input = df_input[2*i:2*i+2]
    starting_time = timeit.default_timer()
    inferences = requests.post(endpoint_url, json={"dataframe_records": rest_input.to_dict(orient='records')}, headers=serving_client.headers).json()
    print(f"Inference time, end 2 end :{round((timeit.default_timer() - starting_time)*1000)}ms")
    print("  "+str(inferences))

Sending requests to https://oregon.cloud.databricks.com/realtime-inference/dbdemos_pcb_classification_endpoint/invocations
Inference time, end 2 end :843ms
  {'predictions': [{'score': 0.9999707937240601, 'label': 'normal'}, {'score': 0.9998716115951538, 'label': 'normal'}]}
Inference time, end 2 end :809ms
  {'predictions': [{'score': 0.9999814033508301, 'label': 'normal'}, {'score': 0.9999306201934814, 'label': 'normal'}]}
Inference time, end 2 end :814ms
  {'predictions': [{'score': 0.999972939491272, 'label': 'normal'}, {'score': 0.9995645880699158, 'label': 'damaged'}]}


## Conclusion

We covered how Databricks makes it easy to deploy deep learning model at scale, including behind a REST endpoint with Databricks Serverless Model Serving.

### Next step: model explainability

As next step, let's discover how to explain and highlight the pixels our model consider as damaged.

Open the [04-explaining-inference notebook]($./04-explaining-inference) to discover how to use SHAP to analyze our prediction.

### Going further

Working with huggingface might not be enough for you. For deeper, custom integration, you can also leverage libraries like pytorch.

Open the [05-torch-lightning-training-and-inference]($./05-torch-lightning-training-and-inference) notebook to discover how to train and deploy a Pytorch model with [PyTorch Lightning](https://www.pytorchlightning.ai/index.html).